In [18]:
import cv2
import numpy as np
# import cardData
import imagehash
import utils
import timeit
# import time
from IPython.display import display, clear_output
import PIL.Image
import io


ModuleNotFoundError: No module named 'imagehash'

In [10]:
import os
print(os.getcwd())


c:\Users\valen\Downloads\Pokemon-Card-Scanner-main\Pokemon-Card-Scanner-main


In [4]:
widthCard = 330
heightCard = 440
cam = cv2.VideoCapture(1)
background_subtractor = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=50, detectShadows=False)

blackImg = np.zeros((heightCard, widthCard, 3), np.uint8)
try:
    while True:

        check, frame = cam.read()

        rot90frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
        # rot90frame = cv2.resize(rot90frame, (widthCard+50, heightCard+50))


        # Make image gray scale
        grayFrame = cv2.cvtColor(rot90frame, cv2.COLOR_BGR2GRAY)
        # Blur the image to reduce noise
        blurredFrame = cv2.GaussianBlur(grayFrame, (3, 3), 0)

        # Background Remover 

        fg_mask = background_subtractor.apply(frame)
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
        fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_OPEN, kernel)
        # foreground = cv2.bitwise_and(blurredFrame, blurredFrame, mask=fg_mask)

        # Use Canny edge detection to get edges
        edgedFrame = cv2.Canny(fg_mask, threshold1=80, threshold2=150)
        # edgedFrame = cv2.adaptiveThreshold(blurredFrame, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 303, 56)
        # Clean up edges
        kernel = np.ones((5,5))
        frameDial = cv2.dilate(edgedFrame, kernel, iterations=2)
        frameThreshold = cv2.erode(frameDial, kernel, iterations=1)

        # Get image contours
        contourFrame = rot90frame.copy()
        bigContour = rot90frame.copy()
        contours, hierarchy = cv2.findContours(edgedFrame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(contourFrame, contours, -1, (0, 255, 0), 10)

        imgWarpColored = blackImg  # Set imgWarpColored
        # Get biggest contour
        corners, maxArea = utils.biggestContour(contours)
        # Initialize values for matching card and found to ensure loop works even though no values are found. 
        matchingCard = np.zeros((heightCard, widthCard, 3), dtype=np.uint8)
        found = False #
        cardinfo = 0
        if len(corners) == 4:
            corners = [corners[0][0], corners[1][0], corners[2][0], corners[3][0]]
            corners = utils.reorderCorners(corners)  # Reorders corners to [topLeft, topRight, bottomLeft, bottomRight]
            #cv2.drawContours(bigContour, corners, -1, (0, 255, 0), 10)
            bigContour = utils.drawRectangle(bigContour, corners)
            pts1 = np.float32(corners)
            pts2 = np.float32([[0, 0], [widthCard, 0], [0, heightCard], [widthCard, heightCard]])
            # Makes a matrix that transforms the detected card to a vertical rectangle
            matrix = cv2.getPerspectiveTransform(pts1, pts2)
            # Transforms card to a rectangle widthCard x heightCard
            imgWarpColored = cv2.warpPerspective(rot90frame, matrix, (widthCard, heightCard))
            section_start = timeit.default_timer()
            # Check if a matching card has been found, and if so, display it
            found, matchingCard, cardinfo = utils.findCard(imgWarpColored.copy())  # Check to see if a matching card was found
            section_end = timeit.default_timer()            

        # Resize all of the images to the same dimensions
        # Note: imgWarpColored is already resized and matchingCard gets resized in utils.getMatchingCard()
        rot90frame = cv2.resize(rot90frame, (widthCard, heightCard))
        grayFrame = cv2.resize(grayFrame, (widthCard, heightCard))
        blurredFrame = cv2.resize(blurredFrame, (widthCard, heightCard))
        edgedFrame = cv2.resize(edgedFrame, (widthCard, heightCard))
        contourFrame = cv2.resize(contourFrame, (widthCard, heightCard))
        bigContour = cv2.resize(bigContour, (widthCard, heightCard))
        fg_mask = cv2.resize(fg_mask, (widthCard, heightCard))

        # print(f"Time to read and rotate image: {section_end - section_start:.4f} seconds")

        # An array of all 8 images
        imageArr = ([rot90frame, grayFrame, blurredFrame, edgedFrame],
                    [contourFrame, bigContour, imgWarpColored, fg_mask])

        # Labels for each image
        labels = [["Original", "Gray", "Blurred", "Threshold"],
                    ["Contours", "Biggest Contour", "Warped Perspective", "foreground"]]

        # Stack all 8 images into one and add text labels
        stackedImage = utils.makeDisplayImage(imageArr, labels)
        imageArr = cv2.cvtColor(stackedImage, cv2.COLOR_BGR2RGB)
        
        # Convert the frame to an image format compatible with Jupyter
        pil_im = PIL.Image.fromarray(imageArr)
        buf = io.BytesIO()
        pil_im.save(buf, format='JPEG')
        
        # Display the frame in Jupyter Notebook
        clear_output(wait=True)
        display(PIL.Image.open(buf))
        # Display the image
except KeyboardInterrupt:
    # This allows you to stop the loop with an interrupt
    print("Stopped by user")
finally:
    # Release the camera and close any open windows
    cam.release()
    cv2.destroyAllWindows() 

NameError: name 'utils' is not defined

In [ ]:
import cv2
from IPython.display import display, clear_output
import PIL.Image
import io

# Open the video capture
cam = cv2.VideoCapture(0)  # Change the argument to 1 if using an external camera

# Set desired width and height
widthCard = 330
heightCard = 440
cam.set(cv2.CAP_PROP_FRAME_WIDTH, widthCard)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, heightCard)

try:
    while True:
        # Capture a frame from the camera
        check, frame = cam.read()
        if not check:
            print("Failed to capture image")
            break

        # Rotate and resize frame if needed
        frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
        frame = cv2.resize(frame, (widthCard, heightCard))

        # Convert to RGB format for display in Jupyter
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Convert the frame to an image format compatible with Jupyter
        pil_im = PIL.Image.fromarray(rgb_frame)
        buf = io.BytesIO()
        pil_im.save(buf, format='JPEG')
        
        # Display the frame in Jupyter Notebook
        clear_output(wait=True)
        display(PIL.Image.open(buf))

except KeyboardInterrupt:
    # This allows you to stop the loop with an interrupt
    print("Stopped by user")
finally:
    # Release the camera and close any open windows
    cam.release()
    cv2.destroyAllWindows() 


Stopped by user


Stopped by user


In [ ]:
widthCard = 330
heightCard = 440
cam = cv2.VideoCapture(1)
background_subtractor = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=50, detectShadows=False)

blackImg = np.zeros((heightCard, widthCard, 3), np.uint8)
try:
    while True:
        check, frame = cam.read()
        if not check:
            print("Camera feed not detected.")
            break

        # Rotate the frame to get the correct orientation
        rot90frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
        
        # Make the image grayscale
        grayFrame = cv2.cvtColor(rot90frame, cv2.COLOR_BGR2GRAY)
        blurredFrame = cv2.GaussianBlur(grayFrame, (3, 3), 0)

        # Apply background subtraction
        fg_mask = background_subtractor.apply(rot90frame)
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
        fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_OPEN, kernel)

        # Edge detection
        edgedFrame = cv2.Canny(fg_mask, threshold1=80, threshold2=150)
        kernel = np.ones((5,5))
        frameDial = cv2.dilate(edgedFrame, kernel, iterations=2)
        frameThreshold = cv2.erode(frameDial, kernel, iterations=1)

        # Find contours
        contours, _ = cv2.findContours(edgedFrame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contourFrame = rot90frame.copy()
        cv2.drawContours(contourFrame, contours, -1, (0, 255, 0), 10)

        # Process biggest contour for card
        corners, maxArea = utils.biggestContour(contours)
        matchingCard = np.zeros((heightCard, widthCard, 3), dtype=np.uint8)
        found, cardinfo = False, 0
        imgWarpColored = blackImg

        if len(corners) == 4:
            corners = utils.reorderCorners([corners[i][0] for i in range(4)])
            bigContour = utils.drawRectangle(rot90frame.copy(), corners)
            pts1 = np.float32(corners)
            pts2 = np.float32([[0, 0], [widthCard, 0], [0, heightCard], [widthCard, heightCard]])
            matrix = cv2.getPerspectiveTransform(pts1, pts2)
            imgWarpColored = cv2.warpPerspective(rot90frame, matrix, (widthCard, heightCard))
            found, matchingCard, cardinfo = utils.findCard(imgWarpColored.copy())

        # Resize frames for display
        rot90frame = cv2.resize(rot90frame, (widthCard, heightCard))
        grayFrame = cv2.resize(grayFrame, (widthCard, heightCard))
        blurredFrame = cv2.resize(blurredFrame, (widthCard, heightCard))
        edgedFrame = cv2.resize(edgedFrame, (widthCard, heightCard))
        contourFrame = cv2.resize(contourFrame, (widthCard, heightCard))
        bigContour = cv2.resize(bigContour, (widthCard, heightCard)) if 'bigContour' in locals() else blackImg
        fg_mask = cv2.resize(fg_mask, (widthCard, heightCard))

        # Stack and display images in Jupyter Notebook
        imageArr = ([rot90frame, grayFrame, blurredFrame, edgedFrame],
                    [contourFrame, bigContour, imgWarpColored, fg_mask])
        labels = [["Original", "Gray", "Blurred", "Threshold"],
                  ["Contours", "Biggest Contour", "Warped Perspective", "Foreground Mask"]]
        
        stackedImage = utils.makeDisplayImage(imageArr, labels)
        stackedImage = cv2.cvtColor(stackedImage, cv2.COLOR_BGR2RGB)
        
        # Convert and display
        pil_im = PIL.Image.fromarray(stackedImage)
        buf = io.BytesIO()
        pil_im.save(buf, format='JPEG')
        
        clear_output(wait=True)
        display(PIL.Image.open(buf))

except KeyboardInterrupt:
    print("Stopped by user")
finally:
    cam.release()
    cv2.destroyAllWindows()


In [19]:
import cv2
import numpy as np
import timeit
from IPython.display import display, clear_output
import PIL.Image
import io

widthCard = 330
heightCard = 440
cam = cv2.VideoCapture(1)
background_subtractor = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=50, detectShadows=False)

blackImg = np.zeros((heightCard, widthCard, 3), np.uint8)

def biggestContour(contours, min_area=10000):
    largest = None
    max_area = 0
    approx_corners = []

    for contour in contours:
        area = cv2.contourArea(contour)
        if area > min_area:  # Only consider contours with area larger than min_area
            perimeter = cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, 0.02 * perimeter, True)

            if len(approx) == 4 and area > max_area:
                largest = contour
                max_area = area
                approx_corners = approx

    return approx_corners, max_area

try:
    while True:
        check, frame = cam.read()
        if not check:
            print("Camera feed not detected.")
            break

        # Rotate the frame to get the correct orientation
        rot90frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
        
        # Make the image grayscale
        grayFrame = cv2.cvtColor(rot90frame, cv2.COLOR_BGR2GRAY)
        blurredFrame = cv2.GaussianBlur(grayFrame, (3, 3), 0)

        # Apply background subtraction
        fg_mask = background_subtractor.apply(rot90frame)
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
        fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_OPEN, kernel)

        # Edge detection
        edgedFrame = cv2.Canny(fg_mask, threshold1=80, threshold2=150)
        kernel = np.ones((5,5))
        frameDial = cv2.dilate(edgedFrame, kernel, iterations=2)
        frameThreshold = cv2.erode(frameDial, kernel, iterations=1)

        # Find contours
        contours, _ = cv2.findContours(edgedFrame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contourFrame = rot90frame.copy()
        cv2.drawContours(contourFrame, contours, -1, (0, 255, 0), 10)

        # Process biggest contour for card
        corners, maxArea = biggestContour(contours)
        matchingCard = np.zeros((heightCard, widthCard, 3), dtype=np.uint8)
        found, cardinfo = False, 0
        imgWarpColored = blackImg

        if len(corners) == 4:
            corners = [corners[i][0] for i in range(4)]
            bigContour = rot90frame.copy()
            cv2.drawContours(bigContour, [np.array(corners)], -1, (0, 255, 0), 10)
            pts1 = np.float32(corners)
            pts2 = np.float32([[0, 0], [widthCard, 0], [0, heightCard], [widthCard, heightCard]])
            matrix = cv2.getPerspectiveTransform(pts1, pts2)
            imgWarpColored = cv2.warpPerspective(rot90frame, matrix, (widthCard, heightCard))
            
            # Assuming utils.findCard is a function that finds the card match, you may keep this line if needed
            # found, matchingCard, cardinfo = utils.findCard(imgWarpColored.copy())

        # Resize frames for display
        rot90frame = cv2.resize(rot90frame, (widthCard, heightCard))
        grayFrame = cv2.resize(grayFrame, (widthCard, heightCard))
        blurredFrame = cv2.resize(blurredFrame, (widthCard, heightCard))
        edgedFrame = cv2.resize(edgedFrame, (widthCard, heightCard))
        contourFrame = cv2.resize(contourFrame, (widthCard, heightCard))
        bigContour = cv2.resize(bigContour, (widthCard, heightCard)) if 'bigContour' in locals() else blackImg
        fg_mask = cv2.resize(fg_mask, (widthCard, heightCard))

        # Stack and display images in Jupyter Notebook
        imageArr = ([rot90frame, grayFrame, blurredFrame, edgedFrame],
                    [contourFrame, bigContour, imgWarpColored, fg_mask])
        labels = [["Original", "Gray", "Blurred", "Threshold"],
                  ["Contours", "Biggest Contour", "Warped Perspective", "Foreground Mask"]]
        
        # Assuming utils.makeDisplayImage creates a labeled display image; otherwise, you can replace it
        stackedImage = utils.makeDisplayImage(imageArr, labels)
        stackedImage = cv2.cvtColor(stackedImage, cv2.COLOR_BGR2RGB)
        
        # Convert and display
        pil_im = PIL.Image.fromarray(stackedImage)
        buf = io.BytesIO()
        pil_im.save(buf, format='JPEG')
        
        clear_output(wait=True)
        display(PIL.Image.open(buf))

except KeyboardInterrupt:
    print("Stopped by user")
finally:
    cam.release()
    cv2.destroyAllWindows()


NameError: name 'utils' is not defined